### Раскомментить и установить расширение

In [12]:
# pip install ifcopenshell 


  Using cached ifcopenshell-0.7.0.231018-py311-none-macosx_11_0_arm64.whl.metadata (8.8 kB)
Using cached ifcopenshell-0.7.0.231018-py311-none-macosx_11_0_arm64.whl (48.9 MB)
Note: you may need to restart the kernel to use updated packages.


In [13]:
# pip install Pillow

  Using cached pillow-10.2.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (9.7 kB)
Using cached pillow-10.2.0-cp312-cp312-macosx_11_0_arm64.whl (3.3 MB)
Note: you may need to restart the kernel to use updated packages.


In [14]:
# pip install pandas

  Using cached pandas-2.2.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (19 kB)
  Using cached numpy-1.26.4-cp312-cp312-macosx_11_0_arm64.whl.metadata (61 kB)
  Using cached python_dateutil-2.8.2-py2.py3-none-any.whl (247 kB)
  Using cached pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached six-1.16.0-py2.py3-none-any.whl (11 kB)
Using cached pandas-2.2.0-cp312-cp312-macosx_11_0_arm64.whl (11.7 MB)
Using cached numpy-1.26.4-cp312-cp312-macosx_11_0_arm64.whl (13.7 MB)
Using cached pytz-2024.1-py2.py3-none-any.whl (505 kB)
Using cached tzdata-2024.1-py2.py3-none-any.whl (345 kB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
import ifcopenshell
import pandas as pd

No stream support: No module named 'lark'
/var/folders/rn/rl1dzql55sd8tt1fhxq5dlzm0000gn/T/ipykernel_13264/333723811.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [24]:
def extract_ifc_properties_to_dataframe(ifc_path, object_type, properties_list):
    # Загрузка IFC файла
    file = ifcopenshell.open(ifc_path)
    
    # Получение всех объектов заданного типа
    objects = file.by_type(object_type)
    
    # Подготовка данных для DataFrame (pandas)
    data = []

    for obj in objects:
        obj_data = {'GlobalId': obj.GlobalId, 'obj_type': obj.get_info()['type']}
        for property_set in obj.IsDefinedBy:
            if property_set.is_a('IfcRelDefinesByProperties'):
                property_set_definition = property_set.RelatingPropertyDefinition
                if property_set_definition.is_a('IfcPropertySet'):
                    for (pset_name, prop_name) in properties_list:
                        if property_set_definition.Name == pset_name:
                            for property in property_set_definition.HasProperties:
                                if property.Name == prop_name and property.is_a('IfcPropertySingleValue'):
                                    # Добавляем значение свойства в данные объекта
                                    if property.NominalValue != None:
                                        obj_data[f'{pset_name}.{prop_name}'] = property.NominalValue.wrappedValue
        data.append(obj_data)

    # Создание DataFrame
    df = pd.DataFrame(data)
    df.set_index('GlobalId', inplace=True)
    
    return df


In [25]:

# Пример использования функции
ifc_path = '12002-81-020601-КМ3.1.3.ifc'
object_type = 'IfcElement'
properties_list = [
    ('Tekla Quantity', 'Weight'),
    ('Идентификация', "Категория ТСС"),
    ('Маркировка', 'Сечение в каталоге Tekla')
]

df = extract_ifc_properties_to_dataframe(ifc_path, object_type, properties_list)
df


,obj_type,Маркировка.Сечение в каталоге Tekla,Tekla Quantity.Weight,Идентификация.Категория ТСС
GlobalId,,,,
0cryUIbA55I9w6zXlNGfwV,IfcBeam,U16P_8240_97,420.60,NaN
0JG_ZbA9T1Q8XB5M4qoezx,IfcBeam,U16P_8240_97,38.02,NaN
2RTzw4jbf0FQmMYmColiZh,IfcMember,PK80X5.0_32931_2015,19.57,NaN
0D$inzAnnDXvy7fR5jDtv5,IfcBeam,PK80X5.0_32931_2015,12.96,NaN
3_Vu0EUSLDoBPiwg2fsVcI,IfcMember,PK80X5.0_32931_2015,21.31,NaN
...,...,...,...,...
2JPJHsQLT1zOPsMqvJDdo9,IfcBeam,U16P_8240_97,70.76,Лестницы
3NKQ$V9k1F3gAKixq4dbqM,IfcBeam,I30H2_57837_2017,547.02,Балки
131ujOmhvA8QUlDsviILBl,IfcBeam,U20P_8240_97,63.91,Балки


In [27]:
df.describe()

,Tekla Quantity.Weight
count,2054.000000
mean,42.984031
std,175.127039
min,0.000000
25%,1.000000
50%,11.280000
75%,23.030000
max,3200.000000


In [26]:
df.to_csv('ifc_extracted_data.csv')